# Quickstart
This notebook follows the step from the [Quickstart section](https://docs.pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html) of the pyTorch offical documentation.

## Imports

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## Getting the data
Using data from: https://github.com/zalandoresearch/fashion-mnist

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Setting up the model

In [4]:
# device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
device = 'cpu'
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Training the model

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

### Helpers

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310740  [   64/60000]
loss: 2.298971  [ 6464/60000]
loss: 2.278631  [12864/60000]
loss: 2.262238  [19264/60000]
loss: 2.235914  [25664/60000]
loss: 2.219275  [32064/60000]
loss: 2.222962  [38464/60000]
loss: 2.197393  [44864/60000]
loss: 2.196592  [51264/60000]
loss: 2.146991  [57664/60000]
Test Error: 
 Accuracy: 45.5%, Avg loss: 2.146724 

Epoch 2
-------------------------------
loss: 2.166819  [   64/60000]
loss: 2.154778  [ 6464/60000]
loss: 2.094540  [12864/60000]
loss: 2.101090  [19264/60000]
loss: 2.039662  [25664/60000]
loss: 1.995262  [32064/60000]
loss: 2.015509  [38464/60000]
loss: 1.943286  [44864/60000]
loss: 1.952309  [51264/60000]
loss: 1.862352  [57664/60000]
Test Error: 
 Accuracy: 57.6%, Avg loss: 1.865837 

Epoch 3
-------------------------------
loss: 1.906589  [   64/60000]
loss: 1.876831  [ 6464/60000]
loss: 1.758462  [12864/60000]
loss: 1.788629  [19264/60000]
loss: 1.665870  [25664/60000]
loss: 1.637557  [32064/600

## Testing the model

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
valid, invalid = [], []
for n, test in enumerate(test_data):
    x, y = test[0], test[1]
    with torch.no_grad():
        x = x.to(device)
        pred = model(x)

        cls_hat = classes[pred[0].argmax(0)]

        [invalid, valid][cls_hat == classes[y]].append(n)
        # predicted, actual = classes[pred[0].argmax(0)], classes[y]
        # print(f'Predicted: "{predicted}", Actual: "{actual}"')

In [16]:
len(valid)

6394

In [17]:
len(invalid)

3606